In [22]:
import pandas as pd
from langdetect import detect

In [2]:
notesData = pd.read_csv("data/notes.tsv", sep="\t")
noteStatusData = pd.read_csv("data/noteStatusHistory.tsv", sep="\t")
print(notesData.shape)
print(noteStatusData.shape)

C:\Users\cdemm\AppData\Local\Temp\ipykernel_712\57774694.py:1: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  notesData = pd.read_csv("data/notes.tsv", sep="\t")
C:\Users\cdemm\AppData\Local\Temp\ipykernel_712\57774694.py:2: DtypeWarning: Columns (10,19) have mixed types. Specify dtype option on import or set low_memory=False.
  noteStatusData = pd.read_csv("data/noteStatusHistory.tsv", sep="\t")


(1540700, 23)
(1678280, 23)


## Creating data subset for annotation

In [35]:
annoNotes = pd.DataFrame(columns=["noteId", "status", "summary", "label1", "label2"])
hCount = 0
uCount = 0
index = 0

for i in range(10000):

    noteId = notesData.iloc[-i]["noteId"]
    classification = notesData.iloc[-i]["classification"]
    believable = notesData.iloc[-i]["believable"]
    harmful = notesData.iloc[-i]["harmful"]
    valDiff = notesData.iloc[-i]["validationDifficulty"]
    summary = notesData.iloc[-i]["summary"]
    
    noteStatus = noteStatusData[noteStatusData["noteId"] == noteId]
    createdAt = noteStatus["createdAtMillis"]
    currentStatusTime = noteStatus["timestampMillisOfCurrentStatus"]
    currentStatus = noteStatus["currentStatus"]

    if currentStatus.empty:
        currentStatus = "NEEDS_MORE_RATINGS"
    else:
        currentStatus = currentStatus.iloc[0]


    if (classification == "MISINFORMED_OR_POTENTIALLY_MISLEADING" and # We are looking for notes that are supposed to be shown under a tweet
        believable not in ["BELIEVABLEBYFEW", "BELIEVABLE_BY_MANY"] and # We don't want notes that use depricated metrics
        harmful not in ["LITTLE*HARM", "CONSIDERABLE*HARM"] and 
        valDiff not in ["EASY", "CHALLENGING"] and
        "http" in summary and # We want notes that have a source in them
        currentStatus != "NEEDS_MORE_RATINGS" and # We want a note that is either rated "helpful" or "not helpful"
        (currentStatusTime.iloc[0] < 1735520580000) and # We want the rating to be at least a day old
        noteId not in [1860687797557895475, 1838219148725514576, 1806261291679731729, 1855052380733050940, 1859390416967106879, 1825898499860476366, 1852145720356429917, 1805946836592566715, 1805943677719548117, 1805937846525534280, 1805937345633358297] and
        detect(summary) == "en"): # We only want notes in english

        if currentStatus == "CURRENTLY_RATED_HELPFUL" and hCount < 100:
            annoNotes.loc[index, "noteId"] = noteId
            annoNotes.loc[index, "status"] = currentStatus
            annoNotes.loc[index, "summary"] = summary
            index += 1
            hCount += 1

        elif currentStatus == "CURRENTLY_RATED_NOT_HELPFUL" and uCount < 100:
            annoNotes.loc[index, "noteId"] = noteId
            annoNotes.loc[index, "status"] = currentStatus
            annoNotes.loc[index, "summary"] = summary
            index += 1
            uCount += 1

    if index >= 201 and hCount >= 100 and uCount >= 100:
        break

annonotes = annoNotes.astype(str)
annoNotes.to_csv("data/annotation.csv")